### Install

In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl
!pip install -q loralib einops
!pip install -q pandas
!pip install gspread
# !pip install wandb
! pip install -q kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.2 MB/s eta 0:00:00


###Env

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
from huggingface_hub import login
login("xxx")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !wandb login
#e4460d24dea96f9c121f12f8aeb7e4176654b332

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import multiprocessing as mp
from types import SimpleNamespace
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer
from torch.nn.parameter import Parameter
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from transformers import get_cosine_schedule_with_warmup
from transformers import AutoConfig, AutoModelForQuestionAnswering

N_CORES = mp.cpu_count()
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

###Context Dataset


####Config directory

In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 /root/.kaggle/kaggle.json

####Zalo

In [ ]:
! kaggle datasets download -d ducnh279/nlp-data --force

  0% 0.00/3.83M [00:00<?, ?B/s]
100% 3.83M/3.83M [00:00<00:00, 142MB/s]


In [ ]:
! unzip /content/nlp-data.zip

Archive:  /content/nlp-data.zip
  inflating: zac2022_testa_only_question.json  
  inflating: zac2022_train_merged_final.json  


In [ ]:
train_df = pd.read_json('/content/zac2022_train_merged_final.json')

In [ ]:
def get_keys(data):
    return [k for k, _ in data.items()]

def has_answer(df):
    return df[df.data.apply(get_keys).apply(lambda x: 'answer' in x)]

def get_context(data):
    return data['text']

def get_question(data):
    return data['question']

def get_answer_position(data):
    return data['short_candidate_start']

def get_answer(data):
    return data['short_candidate']

def question_context(df, i):
    question = df.data.apply(get_question)[i]
    text = test_df.context[i]
    print(
        f'- Question: {question}\n'
        f'- Context: {text}'
    )
questions = has_answer(train_df).data.apply(get_question).reset_index(drop=True)
contexts = has_answer(train_df).data.apply(get_context).reset_index(drop=True)
answers = has_answer(train_df).data.apply(get_answer).reset_index(drop=True)
answer_starts = has_answer(train_df).data.apply(get_answer_position).reset_index(drop=True)
zalo = pd.DataFrame(
    {
        'question': questions,
        'context': contexts,
        'answer': answers,
        'answer_start': answer_starts
    }
)

In [ ]:
zalo

,question,context,answer,answer_start
0,Đất nước nào không có quân đội,"có 23 quốc gia không có lực lượng quân đội, ba...","Costa Rica, Iceland, Panama, Micronesia, Quần ...",53
1,Núi nào cao nhất châu âu,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...,núi Elbrus,73
2,Thành phố nào là thủ phủ của Ai Cập trong đế q...,"Lịch sử . Ai Cập bị La Mã chiếm năm 30 TCN , v...",Alexandria,48
3,Ai là người đứng đầu trong cuộc chống lại chín...,Các nghiên cứu lịch sử cho thấy Cổ Am rất có t...,Lê Chân,65
4,Nền văn hoá Phục Hưng bắt nguồn từ nước nào,"Phục Hưng (tiếng Pháp: ""Renaissance"", , , từ ""...",Ý,205
...,...,...,...,...
4984,vua nào cho phát hành tiền giấy đầu tiên ở việ...,Tiền giấy xuất hiện . Năm 1396 thời Trần Thuận...,Hồ Quý Ly,54
4985,Ai là người phát minh ra máy quay phim,Máy quay phim: Phát minh kỳ diệu của anh em Lu...,anh em Lumière Phương Hiền,37
4986,tên quận nào ở đà nẵng cũng là tên một danh th...,Ngũ Hành Sơn hay núi Non Nước là một danh thắn...,quận Ngũ Hành Sơn,285
4987,trận đánh nào có ý nghĩa quyết định trong khán...,Trận Như Nguyệt là một trận đánh lớn diễn ra ở...,Trận Như Nguyệt,0


####Xsquad

In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/deepmind/xquad/master/xquad.vi.json')
df.drop('version', axis=1, inplace=True)

In [ ]:
def get_paragraph(df):
    return df.data.apply(lambda x: x['paragraphs'])
para = get_paragraph(df)

dfs = []
for p in para:
    p_df = pd.DataFrame(p)
    question_df = p_df['qas'].apply(lambda x: x[0]['question']).to_frame('question')
    start_pos_and_text_df = p_df['qas'].apply(lambda x: x[0]['answers'][0]).apply(pd.Series)
    final_p_df = pd.concat([question_df, p_df, start_pos_and_text_df], axis=1)
    final_p_df.drop('qas', axis=1, inplace=True)
    final_p_df.columns = ['question', 'context', 'answer_start', 'answer']
    dfs.append(final_p_df)
xsquad = pd.concat(dfs, axis=0, ignore_index=True)
xsquad

,question,context,answer_start,answer
0,Đội thủ Panthers đã thua bao nhiêu điểm?,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",30,308
1,Ai thua đội Broncos ở vòng bảng?,Broncos đã đánh bại Pittsburgh Steelers ở vòng...,20,Pittsburgh Steelers
2,Peyton Manning bao nhiêu tuổi khi anh chơi tro...,Peyton Manning trở thành thủ quân (quaterback)...,196,39
3,Lady Gaga đã giành được bao nhiêu giải Grammy?,Người chiến thắng giải Grammy sáu lần và người...,30,sáu
4,Carolina đã bắt đầu ở vạch nào khi trận đấu cò...,"Khi thời gian quy định còn lại 4:51, Carolina ...",61,24
...,...,...,...,...
235,Ai đã đưa ra một cuộc thảo luận triết học về lực?,Aristotle đưa ra bản thảo luận triết học về kh...,0,Aristotle
236,Ai hình thành thuyết về lực vạn vật hấp dẫn?,Sự phát triển của các lý thuyết cơ bản đối với...,143,Isaac Newton
237,Lý thuyết nào giải thích rõ ràng nhất về trọng...,"Kể từ đó đến nay, thuyết tương đối đã được côn...",18,thuyết tương đối
238,Tốc độ thay đổi của điện tích là gì?,Bằng việc kết hợp định nghĩa của dòng diện giố...,33,dòng diện


####Bertqa

In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/mailong25/bert-vietnamese-question-answering/master/dataset/train-v2.0.json')
para = get_paragraph(df)

dfs = []
for p in tqdm(para):
    p_df = pd.DataFrame(p)
    p_df = p_df[p_df.qas.apply(lambda x: x[0]['answers'] != [])].reset_index(drop=True)
    if p_df.values.tolist() == []:
        continue
    question_df = p_df['qas'].apply(lambda x: x[0]['question']).to_frame('question')
    start_pos_and_text_df = p_df['qas'].apply(lambda x: x[0]['answers'][0]).apply(pd.Series)
    final_p_df = pd.concat([question_df, p_df, start_pos_and_text_df], axis=1)
    final_p_df.drop('qas', axis=1, inplace=True)
    final_p_df.columns = ['question', 'context', 'answer_start', 'answer']
    dfs.append(final_p_df)
bert_qa = pd.concat(dfs, axis=0, ignore_index=True)

  0%|          | 0/450 [00:00<?, ?it/s]

In [ ]:
bert_qa

,question,context,answer_start,answer
0,Ai là thủ tướng Ấn Độ hiện tại,Narendra Damodardas Modi ( tiếng Gujarat : નરે...,0,Narendra Damodardas Modi
1,Ai là thủ tướng Ấn Độ hiện tại,- Google hôm 3/6 đã xin lỗi Thủ tướng Ấn Độ Na...,44,Narendra Modi
2,Ai là thủ tướng Ấn Độ hiện tại,"Tin tức về chủ đề "" Thủ tướng Ấn Độ Narendra M...",36,Narendra Modi
3,Ai là chủ tịch tập đoàn Hòa Phát,"Ngày hôm nay , trên các diễn đàn tài chính chứ...",80,Trần Đình Long
4,Ai là chủ tịch tập đoàn Hòa Phát,Chủ tịch Tập đoàn Hoà Phát Trần Đình Long ' hạ...,27,Trần Đình Long
...,...,...,...,...
980,Tổng thư ký NATO là ai,Ngày 30/1 Tổng thư ký NATO Jens Stoltenberg đã...,27,Jens Stoltenberg
981,Ai là thống đốc ngân hàng nhà nước Việt Nam,Ông Nguyễn Văn Giàu ( sinh ngày 8/12/1957 tại ...,4,Nguyễn Văn Giàu
982,Ai là thống đốc ngân hàng nhà nước Việt Nam,Thống đốc Ngân hàng Nhà nước Việt Nam hiện nay...,54,Lê Minh Hưng
983,Ai là thống đốc ngân hàng nhà nước Việt Nam,14 Lê Minh Hưng : Thống đốc Ngân hàng Nhà nước...,3,Lê Minh Hưng


####Merge Data

In [ ]:
qa_dataset = pd.concat([zalo, xsquad, bert_qa], axis=0, ignore_index=True)
qa_dataset.to_json('qa_dataset.json')

In [ ]:
import pandas as pd
from datasets import Dataset
import json

file_path = '/content/qa_dataset.json'
with open(file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
contxt = pd.DataFrame(data)
contxt

,question,context,answer,answer_start
0,Đất nước nào không có quân đội,"có 23 quốc gia không có lực lượng quân đội, ba...","Costa Rica, Iceland, Panama, Micronesia, Quần ...",53
1,Núi nào cao nhất châu âu,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...,núi Elbrus,73
2,Thành phố nào là thủ phủ của Ai Cập trong đế q...,"Lịch sử . Ai Cập bị La Mã chiếm năm 30 TCN , v...",Alexandria,48
3,Ai là người đứng đầu trong cuộc chống lại chín...,Các nghiên cứu lịch sử cho thấy Cổ Am rất có t...,Lê Chân,65
4,Nền văn hoá Phục Hưng bắt nguồn từ nước nào,"Phục Hưng (tiếng Pháp: ""Renaissance"", , , từ ""...",Ý,205
...,...,...,...,...
6209,Tổng thư ký NATO là ai,Ngày 30/1 Tổng thư ký NATO Jens Stoltenberg đã...,Jens Stoltenberg,27
6210,Ai là thống đốc ngân hàng nhà nước Việt Nam,Ông Nguyễn Văn Giàu ( sinh ngày 8/12/1957 tại ...,Nguyễn Văn Giàu,4
6211,Ai là thống đốc ngân hàng nhà nước Việt Nam,Thống đốc Ngân hàng Nhà nước Việt Nam hiện nay...,Lê Minh Hưng,54
6212,Ai là thống đốc ngân hàng nhà nước Việt Nam,14 Lê Minh Hưng : Thống đốc Ngân hàng Nhà nước...,Lê Minh Hưng,3


In [ ]:
contxt = Dataset.from_pandas(contxt)

In [ ]:
contxt

Dataset({
    features: ['question', 'context', 'answer', 'answer_start', '__index_level_0__'],
    num_rows: 6214
})

### Data APEC (Closed for now)

####Preprocess

In [ ]:
import pandas as pd
import json
import re

# Đường dẫn đến tệp Excel
excel_file_path = "/content/drive/MyDrive/FAQ.xlsx"

# Đọc tệp Excel vào DataFrame
df = pd.read_excel(excel_file_path)

# Tạo danh sách chứa các cặp câu hỏi và câu trả lời
qa_pairs = []

# Duyệt qua mỗi hàng trong DataFrame
for index, row in df.iterrows():
    # Lấy nội dung của cột câu hỏi và câu trả lời
    questions = row['QUESTION'].split("\n")  # Tách các câu hỏi trong cell
    response = row['ANSWER']
    questions = [re.sub(r'^\d+\.\s*', '', question) for question in questions]
    questions = [question.rstrip('"') for question in questions]

    # Tạo các cặp câu hỏi và câu trả lời
    for question in questions:
        # Kiểm tra nếu câu hỏi không rỗng thì thêm vào danh sách
        if question.strip():
            qa_pairs.append({"question": question.strip(), "response": response.strip()})

# Ghi danh sách các cặp câu hỏi và câu trả lời vào tệp JSON
output_json_file_path = "/content/drive/MyDrive/FAQ_processed.json"
with open(output_json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(qa_pairs, json_file, ensure_ascii=False, indent=4)

print("Dữ liệu đã được xử lý và ghi vào tệp JSON mới:", output_json_file_path)


Dữ liệu đã được xử lý và ghi vào tệp JSON mới: /content/drive/MyDrive/FAQ_processed.json


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(df['QUESTION'], df['ANSWER'], test_size=0.2, random_state=42)

# Vector hóa văn bản
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Huấn luyện mô hình
model_eval = MultinomialNB()
model_eval.fit(X_train_vec, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model_eval.predict(X_test_vec)

# Độ chính xác
accuracy = accuracy_score(y_test, y_pred)
print("Độ chính xác trên tập kiểm tra:", accuracy*100)

# Precision
precision = precision_score(y_test, y_pred, average='weighted')
print("Precision trên tập kiểm tra:", precision*100)

# Recall
recall = recall_score(y_test, y_pred, average='weighted')
print("Recall trên tập kiểm tra:", recall*100)

# F1-score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score trên tập kiểm tra:", f1*100)


Độ chính xác trên tập kiểm tra: 82.95454545454545
Precision trên tập kiểm tra: 82.90043290043289
Recall trên tập kiểm tra: 82.95454545454545
F1-score trên tập kiểm tra: 81.5196545878364


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


####Read data

In [ ]:
import pandas as pd
from datasets import Dataset
import json

output_json_file_path = "/content/drive/MyDrive/FAQ_processed.json"
json_file_path = output_json_file_path
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# In ra dữ liệu đọc được
print(data)

[{'question': 'Khách sạn có bữa sáng phục vụ tại phòng không?', 'response': 'Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.'}, {'question': 'Làm thế nào để đặt dịch vụ bữa sáng phục vụ tại phòng?', 'response': 'Xin lỗi quý khách, Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.'}, {'question': 'Có phải khách sạn cung cấp bữa sáng tại phòng cho khách không?', 'response': 'Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.'}, {'question': 'Quy trình để yêu cầu bữa sáng tận phòng là gì?', 'response': 'Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.'}, {'question': 'Bữa sáng có chuẩn bị tận phòng không?', 'response': 'Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.'}, {'question': 'Làm thế nào để biết thông tin chi tiết về dịch vụ bữa sáng tận phòng?', 'response': 'Mandala không ph

In [ ]:
my_dict = []
question_responses = data
for qr in question_responses:
    my_dict.append({'text': f"Câu hỏi: {qr['question']}\nTrả lời:{qr['response']}"})

data_set = pd.DataFrame(my_dict)
data = Dataset.from_pandas(data_set)

In [ ]:
len(data['text'])

440

###Config

In [ ]:
cfg = {
    'root': '/content' + '/',
    'seed': 279,
    'batch_size': 16,
    'epochs': 3,
    'weight_decay': 0.001,
    'learning_rate': 5e-5,
    'warmup_steps': 0.0,
    'max_length': 96,
    'intermediate_dropout': 0.0,
    'padding_quantile': 1.0,
    'device': 'cuda',
    'num_workers': N_CORES,
    'backbone': 'xlm-roberta-base'
}

cfg = SimpleNamespace(**cfg)

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def add_end_pos(df):
    len_answers = df.answer.apply(len)
    df['answer_end'] = df.answer_start + len_answers
    return df

def add_token_positions(encodings, start_pos, end_pos):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, start_pos[i]))
        end_positions.append(encodings.char_to_token(i, end_pos[i]-1))
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

@torch.inference_mode()
def run_prediction(context, question):
    inputs = tokenizer.encode_plus(
        question, context,
        return_tensors='pt'
    ).to(cfg.device)

    model.eval()
    with autocast():
        outputs = model(**inputs)

    answer_start = torch.argmax(outputs[0])
    answer_end = torch.argmax(outputs[1]) + 1
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(
            inputs.input_ids[0][answer_start:answer_end]
        )
    )

    print(
        f'- Question: {question}\n'
        f'- Answer: {answer}'
    )

In [ ]:
train_df = pd.read_json(cfg.root + 'qa_dataset.json')

In [ ]:
train_df = add_end_pos(train_df)

In [ ]:
train_df

,question,context,answer,answer_start,answer_end
0,Đất nước nào không có quân đội,"có 23 quốc gia không có lực lượng quân đội, ba...","Costa Rica, Iceland, Panama, Micronesia, Quần ...",53,126
1,Núi nào cao nhất châu âu,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...,núi Elbrus,73,83
2,Thành phố nào là thủ phủ của Ai Cập trong đế q...,"Lịch sử . Ai Cập bị La Mã chiếm năm 30 TCN , v...",Alexandria,48,58
3,Ai là người đứng đầu trong cuộc chống lại chín...,Các nghiên cứu lịch sử cho thấy Cổ Am rất có t...,Lê Chân,65,72
4,Nền văn hoá Phục Hưng bắt nguồn từ nước nào,"Phục Hưng (tiếng Pháp: ""Renaissance"", , , từ ""...",Ý,205,206
...,...,...,...,...,...
6209,Tổng thư ký NATO là ai,Ngày 30/1 Tổng thư ký NATO Jens Stoltenberg đã...,Jens Stoltenberg,27,43
6210,Ai là thống đốc ngân hàng nhà nước Việt Nam,Ông Nguyễn Văn Giàu ( sinh ngày 8/12/1957 tại ...,Nguyễn Văn Giàu,4,19
6211,Ai là thống đốc ngân hàng nhà nước Việt Nam,Thống đốc Ngân hàng Nhà nước Việt Nam hiện nay...,Lê Minh Hưng,54,66
6212,Ai là thống đốc ngân hàng nhà nước Việt Nam,14 Lê Minh Hưng : Thống đốc Ngân hàng Nhà nước...,Lê Minh Hưng,3,15


###Tokenization


In [ ]:
questions = train_df.question.values.tolist()
contexts = train_df.context.values.tolist()
answers = train_df.answer.values.tolist()
answer_starts = train_df.answer_start.values.tolist()
answer_ends = train_df.answer_end.values.tolist()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(cfg.backbone)

encodings = tokenizer(
    contexts, questions,
    padding='max_length',
    truncation=True,
    max_length=cfg.max_length,
)

add_token_positions(
    encodings,
    answer_starts,
    answer_ends
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

###Dataset and dataloader

In [ ]:
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

dataset = QADataset(encodings)
data_loader = DataLoader(
    dataset=dataset,
    batch_size=cfg.batch_size,
    drop_last=True,
    shuffle=True,
    num_workers=cfg.num_workers
)

### Model

In [ ]:
config = AutoConfig.from_pretrained(cfg.backbone)
config.hidden_dropout_prob = cfg.intermediate_dropout
config.attention_probs_dropout_prob = cfg.intermediate_dropout

set_seed(seed=cfg.seed)
model = AutoModelForQuestionAnswering.from_pretrained(
    cfg.backbone,
    config=config
)
model.to(cfg.device);

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = optim.AdamW(
    model.parameters(),
    lr=cfg.learning_rate,
    weight_decay=cfg.weight_decay
)

scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=cfg.warmup_steps,
    num_training_steps=len(data_loader)*cfg.epochs
)

scaler = GradScaler()

### Training

###Merging the LoRA with the base model (dont merge because dont have lora pretrain weight)

In [ ]:
peft_config = LoraConfig(
        r=32,
        lora_alpha=32,
        target_modules='all-linear',
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

In [ ]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
from peft import LoraConfig, get_peft_model
# pretrained_model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=True,
#                             device_map="cuda:0", trust_remote_code=True, use_auth_token=None,)
# tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

peft_model = get_peft_model(model, peft_config)

In [ ]:
peft_model.print_trainable_parameters()

trainable params: 5,333,056 || all params: 282,787,650 || trainable%: 1.8858871665718075


### Train

In [ ]:
for epoch in tqdm(range(cfg.epochs)):

    model.train()

    for batch_idx, batch in enumerate(data_loader):
        input_ids = batch['input_ids'].to(cfg.device)
        attention_mask = batch['attention_mask'].to(cfg.device)
        start_positions = batch['start_positions'].to(cfg.device)
        end_positions = batch['end_positions'].to(cfg.device)

        idx = torch.quantile(
            torch.where(attention_mask==1)[1].float(),
            q=cfg.padding_quantile
        ).long()

        attention_mask = attention_mask[:, :idx]
        input_ids = input_ids[:, :idx]

        with autocast():
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                start_positions=start_positions,
                end_positions=end_positions
            )

            loss = outputs[0]

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        scheduler.step()

        if not batch_idx % 100:
            print(
                f'Epoch: {epoch + 1}/{cfg.epochs}'
                f' | Batch: {batch_idx}/{len(data_loader)}'
                f' | Loss: {loss:.4f}'
            )

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch: 1/3 | Batch: 0/388 | Loss: 4.5296
Epoch: 1/3 | Batch: 100/388 | Loss: 1.0566
Epoch: 1/3 | Batch: 200/388 | Loss: 0.5268
Epoch: 1/3 | Batch: 300/388 | Loss: 0.8896


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch: 2/3 | Batch: 0/388 | Loss: 0.3564
Epoch: 2/3 | Batch: 100/388 | Loss: 0.2086
Epoch: 2/3 | Batch: 200/388 | Loss: 0.4832
Epoch: 2/3 | Batch: 300/388 | Loss: 0.3023
Epoch: 3/3 | Batch: 0/388 | Loss: 0.1345
Epoch: 3/3 | Batch: 100/388 | Loss: 0.2103
Epoch: 3/3 | Batch: 200/388 | Loss: 0.0034
Epoch: 3/3 | Batch: 300/388 | Loss: 0.0303


In [ ]:
new_model_path = "/content/drive/MyDrive/XRoberta_context/qa_model.pth"

In [ ]:
model.eval()
torch.save(model.state_dict(), new_model_path)

###LLM Response

In [ ]:
context = """
Quy định của khách sạn Mandala bao gồm khách hàng không được hút thuốc trong phòng,
khách hàng không mang chó mèo vào khách sạn.
"""
context1 = """
Bùi Quang Khải (sinh ngày 1 tháng 9 năm 2002), là một kỹ sư, nhà tài phiệt,
nhà phát minh, doanh nhân công nghệ và nhà từ thiện người Việt Nam.
"""
context2 = """
khách sạn không cho phép hút thuốc trong phòng"""

In [ ]:
question = "tôi có được hút thuốc trong phòng không"

In [ ]:
question1 = "Bùi Quang Khải sinh ngày bao nhiêu"

In [ ]:
question2 = 'Em trai Khải tên là gì?'

In [ ]:
run_prediction(context2, question)

- Question: tôi có được hút thuốc trong phòng không
- Answer: tôi có được hút thuốc trong phòng không
